### Test a trained model against different attacks

In [1]:
from __future__ import print_function
import os, sys, time, datetime, json, random
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import Conv2D, Flatten, Input
from keras.optimizers import SGD , Adam, RMSprop
from keras.layers.advanced_activations import PReLU
import keras.backend as K
from keras.models import Model
import tensorflow as tf
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
import csv

Using TensorFlow backend.


#### Definition of the maze and its methods

In [2]:
visited_mark = 0.8  # Cells visited by the rat will be painted by gray 0.8
rat_mark = 0.5      # The current rat cell will be painteg by gray 0.5
LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3

# Dict with the posible actions (movements of the mouse)
actions_dict = {
    LEFT: 'left',
    UP: 'up',
    RIGHT: 'right',
    DOWN: 'down',
}

num_actions = len(actions_dict)

# Exploration factor
epsilon = 0.1

In [3]:
# maze is a 2d Numpy array of floats between 0.0 to 1.0
# 1.0 corresponds to a free cell, and 0.0 an occupied cell
# rat = (row, col) initial rat position (defaults to (0,0))

# See https://www.samyzaf.com/ML/rl/qmaze.html for a complete description of the class and how the implementation
# of this CNN works together with reinforcement learning

class Qmaze(object):
    def __init__(self, maze, rat=(0,0)):
        self._maze = np.array(maze)
        nrows, ncols = self._maze.shape
        self.target = (nrows-1, ncols-1) # target cell where the exit is
        self.free_cells = [(r,c) for r in range(nrows) for c in range(ncols) if self._maze[r,c] == 1.0]
        self.free_cells.remove(self.target)
        if self._maze[self.target] == 0.0:
            raise Exception("Invalid maze: target cell cannot be blocked!")
        if not rat in self.free_cells:
            raise Exception("Invalid Rat Location: must sit on a free cell")
        self.reset(rat)

    def reset(self, rat):
        self.rat = rat
        self.maze = np.copy(self._maze)
        nrows, ncols = self.maze.shape
        row, col = rat
        self.maze[row, col] = rat_mark
        self.state = (row, col, 'start')
        self.min_reward = -0.5 * self.maze.size
        self.total_reward = 0
        self.visited = set()

    def update_state(self, action):
        nrows, ncols = self.maze.shape
        nrow, ncol, nmode = rat_row, rat_col, mode = self.state

        if self.maze[rat_row, rat_col] > 0.0:
            self.visited.add((rat_row, rat_col)) # mark visited cell

        valid_actions = self.valid_actions()
                
        if not valid_actions:
            nmode = 'blocked'
        elif action in valid_actions:
            nmode = 'valid'
            if action == LEFT:
                ncol -= 1
            elif action == UP:
                nrow -= 1
            if action == RIGHT:
                ncol += 1
            elif action == DOWN:
                nrow += 1
        else: # invalid action, no change in rat position
            mode = 'invalid'

        # new state
        self.state = (nrow, ncol, nmode)

    def get_reward(self):
        rat_row, rat_col, mode = self.state
        nrows, ncols = self.maze.shape
        if rat_row == nrows-1 and rat_col == ncols-1:
            return 1.0
        if mode == 'blocked':
            return self.min_reward - 1
        if (rat_row, rat_col) in self.visited:
            return -0.25
        if mode == 'invalid':
            return -0.75
        if mode == 'valid':
            return -0.04

    def act(self, action):
        self.update_state(action)
        reward = self.get_reward()
        self.total_reward += reward
        status = self.game_status()
        envstate = self.observe()
        return envstate, reward, status

    def observe(self):
        canvas = self.draw_env()
        envstate = canvas.reshape((7,7,1))
        return envstate

    def draw_env(self):
        canvas = np.copy(self.maze)
        nrows, ncols = self.maze.shape
        # clear all visual marks
        for r in range(nrows):
            for c in range(ncols):
                if canvas[r,c] > 0.0:
                    canvas[r,c] = 1.0
        # draw the rat
        row, col, valid = self.state
        canvas[row, col] = rat_mark
        return canvas

    def game_status(self):
        if self.total_reward < self.min_reward:
            return 'lose'
        rat_row, rat_col, mode = self.state
        nrows, ncols = self.maze.shape
        if rat_row == nrows-1 and rat_col == ncols-1:
            return 'win'

        return 'not_over'

    def valid_actions(self, cell=None):
        if cell is None:
            row, col, mode = self.state
        else:
            row, col = cell
        actions = [0, 1, 2, 3]
        nrows, ncols = self.maze.shape
        if row == 0:
            actions.remove(1)
        elif row == nrows-1:
            actions.remove(3)

        if col == 0:
            actions.remove(0)
        elif col == ncols-1:
            actions.remove(2)

        if row>0 and self.maze[row-1,col] == 0.0:
            actions.remove(1)
        if row<nrows-1 and self.maze[row+1,col] == 0.0:
            actions.remove(3)

        if col>0 and self.maze[row,col-1] == 0.0:
            actions.remove(0)
        if col<ncols-1 and self.maze[row,col+1] == 0.0:
            actions.remove(2)

        return actions

In [4]:
# Print the current status of the maze
def show(qmaze):
    plt.grid('on')
    nrows, ncols = qmaze.maze.shape
    ax = plt.gca()
    ax.set_xticks(np.arange(0.5, nrows, 1))
    ax.set_yticks(np.arange(0.5, ncols, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    canvas = np.copy(qmaze.maze)
    for row,col in qmaze.visited:
        canvas[row,col] = 0.6
    rat_row, rat_col, _ = qmaze.state
    canvas[rat_row, rat_col] = 0.3 # rat cell
    canvas[nrows-1, ncols-1] = 0.9 # exit cell
    img = plt.imshow(canvas, interpolation='none', cmap='gray')
    return img

In [5]:
# Simulate/evaluate a complete game
def play_game(model, qmaze, rat_cell):
    start_time = datetime.datetime.now()
    qmaze.reset(rat_cell)
    envstate = qmaze.observe()
    movements = 0
    actions =""
    while True:
        #show(qmaze)
        movements= movements + 1
        prev_envstate = envstate
        # get next action
        q = model.predict(prev_envstate.reshape(1,7,7,1))
        action = np.argmax(q[0])
        actions = actions +","+actions_dict[action]
        #print(q[0], actions_dict[action])
        #print(actions_dict[action], end=',')
        
        # apply action, get rewards and new state
        envstate, reward, game_status = qmaze.act(action)
        if game_status == 'win':
            dt = datetime.datetime.now() - start_time
            #print("time:%s, movements:%s" % (dt, movements))
            return True, dt, movements, actions
        elif game_status == 'lose':
            dt = datetime.datetime.now() - start_time
            #print("time:%s, movements:%s" % (dt, movements))
            return False, dt, movements, actions

In [6]:
# Simulate/evaluate a complete game
def play_game_jamming(model_original, model_attack, number_positions_attack, is_change_model, qmaze, rat_cell):
    start_time = datetime.datetime.now()
    qmaze.reset(rat_cell)
    envstate = qmaze.observe()
    movements = 0
    actions =""
    position_counter = 1
    while True:
        #show(qmaze)
        movements= movements + 1
        prev_envstate = envstate
        # get next action
        
        if is_change_model:
            if position_counter <= number_positions_attack: # Use the original model until reaching number_positions_attack
                model = model_attack
                #print("modelo ataque")
                #print("position_counter: ", str(position_counter))
            else:
                model = model_original
                #print("modelo normal")
        else:
            model = model_attack # Always use the attacked model
        
        q = model.predict(prev_envstate.reshape(1,7,7,1))
        action = np.argmax(q[0])
        actions = actions +","+actions_dict[action]
        #print(q[0], actions_dict[action])
        #print(actions_dict[action], end=',')
        
        # apply action, get rewards and new state
        envstate, reward, game_status = qmaze.act(action)
        if game_status == 'win':
            dt = datetime.datetime.now() - start_time
            #print("time:%s, movements:%s" % (dt, movements))
            return True, dt, movements, actions
        elif game_status == 'lose':
            dt = datetime.datetime.now() - start_time
            #print("time:%s, movements:%s" % (dt, movements))
            return False, dt, movements, actions
        
        position_counter += 1

In [7]:
# We simulate all possible games and check if our model wins them all (only for small mazes)
def completion_check(model, qmaze):
    for cell in qmaze.free_cells:
        if not qmaze.valid_actions(cell):
            return False
        if not play_game(model, qmaze, cell):
            return False
    return True

In [8]:
'''
Dump data to a CSV file

file_name: path/name of the file to write
line: array to write (each position is written in a different column, within the same row)
'''
def write_to_csv_file(file_name, line, mode):
    with open(file_name, mode) as csv_file:
        writer = csv.writer(csv_file, delimiter=';')
        writer.writerow(line)

#### Method to modify the matrix of weights (simulate an attack)

In [9]:
'''
Masking function factory
It modifies the output of the target node of the network, by multiplying its output by "percentage"

xyf_coord = [x,y,filter]; xyf_size = [x_size, y_size, filter_size]
'''
def mask_img(xyf_coord, xyf_size, percentage, bias):
    x_size, y_size, f_size = xyf_size # extract the three components
    if y_size is None:
        y_size = x_size
    # Masking function
    def mask_func(img):
        img_shape = K.shape(img)
        # Make indexing arrays
        # nn = K.arange(img_shape[0])
        xx = K.arange(img_shape[1]) # 3
        yy = K.arange(img_shape[2]) # 3
        ff = K.arange(img_shape[3]) # 8 (en el ejemplo)

        # Get coordinates
        xyf = K.cast(K.constant(xyf_coord), img_shape.dtype)
        x = xyf[0:1]
        y = xyf[1:2]
        f = xyf[2:3]
        
        # Make X and Y masks: (example: [1,2,3] [1,1,1])
        mask_x = (xx >= x) & (xx < x + x_size) # xx=[0,1,2] -> [F,T,T]; [T,T,F] -> [F,T,F] isolates the "1"
        mask_y = (yy >= y) & (yy < y + y_size) # yy=[0,1,2] -> [F,F,T]; [T,T,T] -> [F,F,T] isolates the "2"
        mask_f = (ff >= f) & (ff < f + f_size) # ff =[0,1,2,3,4,5,6,7] -> [F,F,F,T,T,T,T,T]; [T,T,T,T,F,F,F,F] -> [F,F,F,T,F,F,F,F] isolates the "3"
        
        # Now we have in binary: mask_x = [0,1,0]; mask_y=[0,0,1]; mask_f=[0,0,0,1,0,0,0,0]
    
        # Make full mask
        # I want to get: [[[0]],[[1]],[[0]]]
        mask_x_expanded = K.expand_dims(K.expand_dims(mask_x, -1), -1)
        # After the first expand: [0,1,0] -> [[0],[1],[0]] (3,1)
        # After the second expand: [[[0]],[[1]],[[0]]] (3,1,1)
        mask_y_expanded = K.expand_dims(K.expand_dims(mask_y, 0), -1)
        # After the first expand:[0,0,1] -> [[0,0,1]] (1,3)
        # After the second expand: [[[0],[0],[1]]] (1,3,1)
        mask_f_expanded = K.expand_dims(K.expand_dims(mask_f, 0), 0)
        # Result after the process: [[[0,0,0,1,0,0,0,0]]]  (1,1,8)
        mask = mask_x_expanded & mask_y_expanded & mask_f_expanded # (3,3,8)
        # Add channels dimension
        mask = K.expand_dims(mask, 0)  # (1,3,3,8)
        # Multiply image and mask
        mask = K.cast(mask, img.dtype)

        return img + img * mask * percentage + bias * mask
    return mask_func

# Model -> Example to test the method
img = Input(shape=(5, 5, 8))  # Small size for test
# We mask from position [0,0,0] a volume of 3 vertical elements,
# 2 horizontals and 1 filter in depth
output = Lambda(mask_img([4,4,0], [1, 1, 1], -1, 0.0 ))(img)
model = Model(inputs=[img], outputs=output)
#out = model.predict("an image of size 1,5,5,8)

In [ ]:
'''
It exchanges the output of two subarrays with the same saze

xyf1_coord = [x1, y1, filter]; xyf2_coord = [x2, y2, filter]; xyf_size = [x_size, y_size, filter_size]
'''

def mask_img_xchg(xyf1_coord, xyf2_coord, xyf_size):
    x_size, y_size, f_size = xyf_size  # extract the three components

    # Masking function
    def mask_func(img):
        def generate_masks(img, xyf_coord, x_size, y_size, f_size):
            img_shape = K.shape(img)
            # Make indexing arrays
            xx = K.arange(img_shape[1])
            yy = K.arange(img_shape[2])
            ff = K.arange(img_shape[3])

            # Get coordinates
            xyf = K.cast(K.constant(xyf_coord), img_shape.dtype)
            x = xyf[0:1]
            y = xyf[1:2]
            f = xyf[2:3]

            # Make X and Y masks:
            mask_x = (xx >= x) & (xx < x + x_size)
            mask_y = (yy >= y) & (yy < y + y_size)
            mask_f = (ff >= f) & (ff < f + f_size)
            # Make full mask
            mask_x_expanded = K.expand_dims(K.expand_dims(mask_x, -1), -1)
            mask_y_expanded = K.expand_dims(K.expand_dims(mask_y, 0), -1)
            mask_f_expanded = K.expand_dims(K.expand_dims(mask_f, 0), 0)
            mask = mask_x_expanded & mask_y_expanded & mask_f_expanded
            # Add channels dimension
            mask = K.expand_dims(mask, 0)
            # Multiply image and mask
            mask = K.cast(mask, img.dtype)
            
            return mask
        
        mask1 = generate_masks(img, xyf1_coord, x_size, y_size, f_size)
        mask2 = generate_masks(img, xyf2_coord, x_size, y_size, f_size)
        
        # value1 = img[x1,y1,f1]
        # value2 = img[x2,y2,f2]
        value1 = img[:,xyf1_coord[0], xyf1_coord[1], xyf1_coord[2]]
        value1 = K.expand_dims(K.expand_dims(K.expand_dims(value1,-1),-1),-1)
        value2 = img[:,xyf2_coord[0], xyf2_coord[1], xyf2_coord[2]]
        value2 = K.expand_dims(K.expand_dims(K.expand_dims(value2,-1),-1),-1)
        
        return img - mask1*value1 + value2*mask1 - value2*mask2 + value1*mask2
    
    return mask_func

In [ ]:
'''
It mirrors the output of two subarrays with the same size

A value in the range [m, M] is mirrored by the middle point of the range. m and M are the input minimum 
and maximum respectively.

new_value = m + M - old_value

xyf_coord = [x1, y1, filter]; xyf_size = [x_size, y_size, filter_size]
'''
def mask_img_mirror(xyf_coord, xyf_size):
    x_size, y_size, f_size = xyf_size  # extract the three components

    # Masking function
    def mask_func(img):
        img_shape = K.shape(img)
        # Make indexing arrays
        # nn = K.arange(img_shape[0])
        xx = K.arange(img_shape[1])  # 3
        yy = K.arange(img_shape[2])  # 3
        ff = K.arange(img_shape[3])  # 8 (en el ejemplo)

        # Get coordinates
        xyf = K.cast(K.constant(xyf_coord), img_shape.dtype)
        x = xyf[0:1]
        y = xyf[1:2]
        f = xyf[2:3]
        
        # Make X and Y masks: (ejemplo: [1,2,3] [1,1,1])
        mask_x = (xx >= x) & (xx < x + x_size)
        mask_y = (yy >= y) & (yy < y + y_size)
        mask_f = (ff >= f) & (ff < f + f_size)
        
        mask_x_expanded = K.expand_dims(K.expand_dims(mask_x, -1), -1)
        mask_y_expanded = K.expand_dims(K.expand_dims(mask_y, 0), -1)
        mask_f_expanded = K.expand_dims(K.expand_dims(mask_f, 0), 0)
        
        mask = mask_x_expanded & mask_y_expanded & mask_f_expanded # (3,3,8)

        mask = K.expand_dims(mask, 0)  # (1,3,3,8)

        mask = K.cast(mask, img.dtype)

        maximum = K.max(img, axis=[1,2,3])
        maximum = K.expand_dims(K.expand_dims(K.expand_dims(maximum,-1),-1),-1)
        minimum = K.min(img, axis=[1,2,3])
        minimum = K.expand_dims(K.expand_dims(K.expand_dims(minimum,-1),-1),-1)
        return (maximum + minimum)*mask - img*mask + img*(1-mask)
    return mask_func

#### Generate the model without training

In [10]:
'''
Given a list of nodes, it first builds an empty model with the size of the first layer of the CNN.
After that, it calls "mask_img" method to apply the attack, focused on modifying the output of the
targeted nodes. After that, it finishes building the "skeleton" of the empty model and compiles it. 
'''
def build_model(maze, nodes, stim_value, bias, lr=0.001): # lr: learning rate. Default 0.001
    row, col = maze.shape
    model = Sequential()
    model.add(Conv2D(8, kernel_size=3, activation='relu', input_shape=(row, col, 1)))  # input_shape = (maze.size,)
    # 7x7 -> 5x5
    
    # Create a mask for each neuron indicated in "nodes" (3D coordinates) and add it to the model
    # volumen [1,1,1] to indicate one neuron in the filter each time 
    for node in nodes:
        model.add(Lambda(mask_img(node, [1,1,1], stim_value, bias)))
    
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    # 5x5 -> 3x3
    model.add(Flatten())
    model.add(Dense(num_actions)) # Dense layer with 4 nodes
    model.compile(optimizer='adam', loss='mse')
    return model

In [11]:
'''
It only creates the empty model
'''
def build_model_without_attack(maze, lr=0.001):
    row, col = maze.shape
    model = Sequential()
    model.add(Conv2D(8, kernel_size=3, activation='relu', input_shape=(row, col, 1)))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_actions))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
'''
It applies an attack based on exchaging the output of pairs of nodes
'''
def build_model_exchange(maze, pairs_nodes, lr=0.001):
    row, col = maze.shape
    model = Sequential()
    model.add(Conv2D(8, kernel_size=3, activation='relu', input_shape=(row, col, 1)))  
    
    for pair in pairs_nodes:
        model.add(Lambda(mask_img_xchg(pair[0], pair[1], [1,1,1])))
    
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_actions))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
'''
It applies an attack based on mirroring the output value of the targeted nodes
'''
def build_model_mirror(maze, nodes, lr=0.001):
    row, col = maze.shape
    model = Sequential()
    model.add(Conv2D(8, kernel_size=3, activation='relu', input_shape=(row, col, 1)))
    
    for node in nodes:
        model.add(Lambda(mask_img_mirror(node, [1,1,1])))
    
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_actions))
    model.compile(optimizer='adam', loss='mse')
    return model

In [3]:
# Definition of the maze
maze =  np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  0.,  1.,  0.],
    [ 0.,  0.,  0.,  1.,  1.,  1.,  0.],
    [ 1.,  1.,  1.,  1.,  0.,  0.,  1.],
    [ 1.,  0.,  0.,  0.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.]
])

NameError: name 'np' is not defined

#### Methods that implement the attacks

In [13]:
'''
It performs the attack over an empty model. 
After that, it loads the original weights of the trained model and calls the method
"play_game" to evaluate the game
'''
def play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details):
    games = games + 1
    qmaze = Qmaze(maze)
    #show(qmaze)
    model = build_model(maze, selected_nodes, stim_value, 0.0)
    model.load_weights("CNN.h5")

    weights = model.get_weights()
    result, time, movements, actions = play_game(model, qmaze,(x,y))

    #line = "--"+str(games)+", "+str(result)+", "+str(time)+", "+str(movements)+", Neurons: "+str(selected_nodes)+", Actions: "+actions+"\n"

    #file_details.write(line)
    #file_details.flush()

    if result == False:
        lose = lose +1 

    del model
    K.clear_session() # required to free the memory of the model
    
    return games, lose, result, movements, time, actions

In [14]:
'''
Perform the attack over the number of positions indicated by number_positions_attack
'''
def play_game_attacks_Jamming(x, y, games, lose, number_positions_attack, is_change_model, selected_nodes, stim_value, file_details):
    games = games + 1
    qmaze = Qmaze(maze)
    #show(qmaze)
    
    model_original = build_model_without_attack(maze, 0.0)
    model_original.load_weights("CNN.h5")
    
    model_attack = build_model(maze, selected_nodes, stim_value, 0.0)
    model_attack.load_weights("CNN.h5")

    #weights = model.get_weights()
    
    result, time, movements, actions = play_game_jamming(model_original, model_attack, number_positions_attack, is_change_model, qmaze,(x,y))
    
    #line = "--"+str(games)+", "+str(result)+", "+str(time)+", "+str(movements)+", Neurons: "+str(selected_nodes)+", Actions: "+actions+"\n"

    #file_details.write(line)
    #file_details.flush()

    if result == False:
        lose = lose +1 

    del model_original
    del model_attack
    K.clear_session() # required to free the memory of the model
    
    return games, lose, result, movements, time

In [15]:
'''
Builds a model without attacks and plays the game
'''
def play_game_without_attacks(x, y, games, lose):
    games = games + 1
    qmaze = Qmaze(maze)
    #show(qmaze)
    model = build_model_without_attack(maze, 0.0)
    model.load_weights("CNN.h5")

    weights = model.get_weights()
    result, time, movements, actions = play_game(model, qmaze,(x,y))
    
    if result == False:
        lose = lose +1 

    del model
    K.clear_session()
    
    return games, lose, result, movements, time

In [ ]:
'''
Performs an exchange attack and plays the game
'''
def play_game_attacks_exchange(x, y, games, lose, selected_nodes, file_details):
    games = games + 1
    qmaze = Qmaze(maze)
    #show(qmaze)
    model = build_model_exchange(maze, selected_nodes)
    model.load_weights("CNN.h5")

    weights = model.get_weights()
    result, time, movements, actions = play_game(model, qmaze,(x,y))

    line = "--"+str(games)+", "+str(result)+", "+str(time)+", "+str(movements)+", Neurons: "+str(selected_nodes)+", Actions: "+actions+"\n"

    file_details.write(line)
    file_details.flush()

    if result == False:
        lose = lose +1 

    del model
    K.clear_session()  # necesario para liberar la memoria del modelo
    
    return games, lose, result, movements, time

In [ ]:
'''
Performs a mirror attack and plays the game
'''
def play_game_attacks_mirror(x, y, games, lose, selected_nodes, file_details):
    games = games + 1
    qmaze = Qmaze(maze)
    #show(qmaze)
    model = build_model_mirror(maze, selected_nodes)
    model.load_weights("CNN.h5")

    weights = model.get_weights()
    result, time, movements, actions = play_game(model, qmaze,(x,y))

    line = "--"+str(games)+", "+str(result)+", "+str(time)+", "+str(movements)+", Neurons: "+str(selected_nodes)+", Actions: "+actions+"\n"

    file_details.write(line)
    file_details.flush()

    if result == False:
        lose = lose +1 

    del model
    K.clear_session()
    
    return games, lose, result, movements, time

In [16]:
# Constants for the number of nodes
N_NODES_ATTACKED = 105
#N_NODES_ATTACKED = 20
#list_stim_values_stimulation = [0.25, 0.5, 0.75, 1.0]

'''
% over the Izhikevich range of values for the "v" parameter, for each overstimulation voltage considered
range = [-65, +30] -> total range of 65mV

10mV -> 10/65 = 0.15
20mV -> 20/65 = 0.3
40mV -> 40/65 = 0.6
60mV -> 60/65 = 0.9
'''
list_stim_values_stimulation = [0.15, 0.3, 0.6, 0.9]

list_stim_values_inhibition = [-1.0]

# [attack_name, n_simultaneous_nodes, list_stim_values]
attacks_dict = {
    'FLO': ['Flooding', N_NODES_ATTACKED, list_stim_values_stimulation], 
    'JAM': ['Jamming', N_NODES_ATTACKED, list_stim_values_inhibition],
    'SCA': ['PortScanning', 1, list_stim_values_stimulation],
    'FOR': ['SelectiveForwarding', 1, list_stim_values_inhibition],
    'SPO': ['Spoofing', int(N_NODES_ATTACKED/2)],
    'SYB': ['Sybil', N_NODES_ATTACKED],
}

header = ['attack', 'n_nodes', 'stim_value', 'X', 'Y', 'position', 'n_steps', 'time', 'solution', 'n_exec', 'attacked_nodes']
header2 = ['attack', 'n_nodes', 'X', 'Y', 'position', 'n_steps', 'time', 'solution', 'n_exec', 'attacked_nodes']

header_jamming = ['attack', 'n_nodes', 'stim_value', 'positions_attacked', 'n_steps', 'time', 'solution', 'n_exec', 'attacked_nodes']

list_coordinates_optimal_path = [[0,0], [1,0], [1,1], [1,2], [0,2], [0,3], [0,4], [0,5], [1,5], [2,5], 
                                 [2,4], [2,3], [3,3], [3,2], [3,1], [3,0], [4,0], [5,0], [6,0], [6,1], 
                                 [6,2], [5,2], [5,3], [5,4], [5,5], [6,5], [6,6]]

#### Method that tests a previously trained model against different attacks, based on different parameters

In [ ]:
'''
attack: string with the name of the attack
initial_node: index defining the number of the first attacked node
step_node: increment in the index of the attacked nodes used to adapt the size of the experiments
end_node: highest attacked index of a node
n_exec: number of executions of each particular combination of parameters
'''
def attack_manager(attack, initial_node, step_node, n_exec):
    X_AXIS = 5
    Y_AXIS = 5
    Z_AXIS = 8 # number of filters
    X_MAZE = 7
    Y_MAZE = 7
    
    selected_nodes = [] # nodes selected to be attacked
    
    if attack in attacks_dict:
        dir_name = '/data/CNN/output/'
        
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            
        file_attack_name =  dir_name + 'attack' + attacks_dict[attack][0] + '.csv'
        file_details_name = dir_name + 'attack' + attacks_dict[attack][0] + '_details.txt'
        file_summary_name = dir_name + 'attack' + attacks_dict[attack][0] + '_summary.txt'

        # Open files
        file_details = open(file_details_name,"w")   
        file_summary = open(file_summary_name,"w")
        
        # Write header (overwrites the file)
        write_to_csv_file(file_attack_name, header, 'w')
        
        # Each number of neurons to consider
        for current_nodes in range(initial_node, attacks_dict[attack][1]+1, step_node):

            #caption= "********* Number of nodes under attack: "+str(current_nodes)+" *********\n\n"
            #file_details.write(caption)
            #file_details.flush()     

            # Each position of the input maze
            #for x in range (X_MAZE):
            #    for y in range (Y_MAZE):
            #position_index = 1
            
            for position in list_coordinates_optimal_path:
                x = position[0]
                y = position[1]
                # Check if the current position of the maze is free (avoid walls)
                if maze[x][y]==1:
                    games=0
                    lose=0

                    if attack not in ['SPO', 'SYB']: # SCA, FOR, FLO, JAM
                        # Each stimulation value
                        for stim_value in attacks_dict[attack][2]:

                            #caption= "********* Starting games from position: ("+str(x)+","+str(y)+") | stim_value: "+str(stim_value)+" | n_nodes: "+str(current_nodes)+" *********\n"
                            #file_details.write(caption)
                            #file_details.flush()

                            if attack in ['SCA', 'FOR']: # attack each node sequentially, one per step of the loop
                                for i in range (X_AXIS):
                                    for j in range (Y_AXIS):  
                                        for k in range (Z_AXIS):
                                            # Store the selected nodes
                                            selected_nodes.append([i, j, k])

                                            # Play the game with the current params
                                            games, lose, result, movements, time = play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details)

                                            # Update the attack file
                                            write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), x, y, str(position_index), str(movements), str(time), str(result), str(1), str(selected_nodes)], 'a')

                                            # Restart variables for next iterations
                                            selected_nodes = []

                            else: # attack multiple random neurons simultaneously
                                # Number of test repetitions indicated by n_exec
                                for current_exec in range(0, n_exec):

                                    # Generate random 3D coordinates for the current number of nodes
                                    for current_random in range(0, current_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                        # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
                                        while ([randX, randY, randZ] in selected_nodes):
                                            randX = np.random.randint(0, X_AXIS)
                                            randY = np.random.randint(0, Y_AXIS)
                                            randZ = np.random.randint(0, Z_AXIS)

                                        # Store the selected node
                                        selected_nodes.append([randX, randY, randZ])

                                    # Play the game with the current params
                                    
                                    #games=10
                                    #lose=1
                                    #result=True
                                    #movements=26
                                    #time=11111
                                    
                                    games, lose, result, movements, time = play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details)
                                    
                                    # movements (number of steps) from the current position of the mouse
                                    # we need to add the steps until reaching that position
                                    movements += (position_index-1)
                                    
                                    # Update the attack file
                                    write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), x, y, str(position_index), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                    # Restart variables for next iterations
                                    selected_nodes = []

                            #file_details.write("=== End games from position: ("+str(x)+","+str(y)+"), WIN: "+str(games-lose)+"\n\n\n")
                            #file_details.flush()
                            #file_summary.write("=== Games from position: ("+str(x)+","+str(y)+") with increment of: "+str(stim_value)+", NODES: "+str(current_nodes)+", PLAYED: "+str(games)+", WINNERS: "+str(games-lose)+" ===\n\n\n")
                            #file_summary.flush()

                    else: # SPO, SYB
                        selected_pairs_nodes = [] # pairs of nodes to be exchanged

                        # Number of test repetitions indicated by n_exec
                        for current_exec in range(0, n_exec):
                            if attack in ['SPO']:
                                # Generate random 3D coordinates for the current number of nodes -> pairs of nodes
                                for current_random in range(0, current_nodes):
                                    # Node 1 of the pair
                                    randX1 = np.random.randint(0, X_AXIS)
                                    randY1 = np.random.randint(0, Y_AXIS)
                                    randZ1 = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates for node 1
                                    while ([randX1, randY1, randZ1] in selected_nodes):
                                        randX1 = np.random.randint(0, X_AXIS)
                                        randY1 = np.random.randint(0, Y_AXIS)
                                        randZ1 = np.random.randint(0, Z_AXIS)

                                    selected_nodes.append([randX1, randY1, randZ1])

                                    # Node 2 of the pair
                                    randX2 = np.random.randint(0, X_AXIS)
                                    randY2 = np.random.randint(0, Y_AXIS)
                                    randZ2 = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates for node 2
                                    while ([randX2, randY2, randZ2] in selected_nodes):
                                        randX2 = np.random.randint(0, X_AXIS)
                                        randY2 = np.random.randint(0, Y_AXIS)
                                        randZ2 = np.random.randint(0, Z_AXIS)

                                    selected_nodes.append([randX2, randY2, randZ2])

                                    # Store the selected neuron
                                    selected_pairs_nodes.append([[randX1, randY1, randZ1], [randX2, randY2, randZ2]])

                                # Play the game with the current params
                                games, lose, result, movements, time = play_game_attacks_exchange(x, y, games, lose, selected_pairs_nodes, file_details)
                                
                                # movements (number of steps) from the current position of the mouse
                                # we need to add the steps until reaching that position
                                movements += (position_index-1)
                                    
                                # Update the attack file
                                write_to_csv_file(file_attack_name, [attack, current_nodes, x, y, str(position_index), str(movements), str(time), str(result), str(current_exec), str(selected_pairs_nodes)], 'a')

                                # Restart variables for next iterations
                                selected_nodes = []
                                selected_pairs_nodes = []

                            else:
                                # Generate random 3D coordinates for the current number of nodes
                                for current_random in range(0, current_nodes):

                                    randX = np.random.randint(0, X_AXIS)
                                    randY = np.random.randint(0, Y_AXIS)
                                    randZ = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates
                                    while ([randX, randY, randZ] in selected_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                    # Store the selected node
                                    selected_nodes.append([randX, randY, randZ])

                                # Play the game with the current params
                                games, lose, result, movements, time = play_game_attacks_mirror(x, y, games, lose, selected_nodes, file_details)
                                
                                # movements (number of steps) from the current position of the mouse
                                # we need to add the steps until reaching that position
                                movements += (position_index-1)
                                    
                                # Update the attack file
                                write_to_csv_file(file_attack_name, [attack, current_nodes, x, y, str(position_index), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                # Restart variables for next iterations
                                selected_nodes = []    

                        #file_details.write("=== End games from position: ("+str(x)+","+str(y)+"), WIN: "+str(games-lose)+"\n\n\n")
                        #file_details.flush()
                        #file_summary.write("=== Games from position: ("+str(x)+","+str(y)+"), NODES: "+str(current_nodes)+", PLAYED: "+str(games)+", WINNERS: "+str(games-lose)+" ===\n\n\n")
                        #file_summary.flush()
                
                # Increment the index used to label the position of the optimal path
                position_index += 1
        #file_details.close()
        #file_summary.close()

In [ ]:
######### FLO: UPDATE POSITIONS AS PARAMETER #########
'''
attack: string with the name of the attack
initial_node: index defining the number of the first attacked node
step_node: increment in the index of the attacked nodes used to adapt the size of the experiments
end_node: highest attacked index of a node
n_exec: number of executions of each particular combination of parameters
'''
def attack_manager(attack, initial_node, step_node, n_exec, list_positions_attack):
    X_AXIS = 5
    Y_AXIS = 5
    Z_AXIS = 8 # number of filters
    X_MAZE = 7
    Y_MAZE = 7
    
    selected_nodes = [] # nodes selected to be attacked
    
    if attack in attacks_dict:
        dir_name = '/data/CNN/output/'
        
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            
        file_attack_name =  dir_name + 'attack' + attacks_dict[attack][0] + '.csv'
        file_details_name = dir_name + 'attack' + attacks_dict[attack][0] + '_details.txt'
        file_summary_name = dir_name + 'attack' + attacks_dict[attack][0] + '_summary.txt'

        # Open files
        file_details = open(file_details_name,"w")   
        file_summary = open(file_summary_name,"w")
        
        # Write header (overwrites the file)
        write_to_csv_file(file_attack_name, header, 'w')
        
        # Each number of neurons to consider
        for current_nodes in range(initial_node, attacks_dict[attack][1]+1, step_node):

            #caption= "********* Number of nodes under attack: "+str(current_nodes)+" *********\n\n"
            #file_details.write(caption)
            #file_details.flush()     

            # Each position of the input maze
            #for x in range (X_MAZE):
            #    for y in range (Y_MAZE):

            for position_index in list_positions_attack:
                x = list_coordinates_optimal_path[position_index][0]
                y = list_coordinates_optimal_path[position_index][1]
                
                # Check if the current position of the maze is free (avoid walls)
                if maze[x][y]==1:
                    games=0
                    lose=0

                    if attack not in ['SPO', 'SYB']: # SCA, FOR, FLO, JAM
                        # Each stimulation value
                        for stim_value in attacks_dict[attack][2]:

                            #caption= "********* Starting games from position: ("+str(x)+","+str(y)+") | stim_value: "+str(stim_value)+" | n_nodes: "+str(current_nodes)+" *********\n"
                            #file_details.write(caption)
                            #file_details.flush()

                            if attack in ['SCA', 'FOR']: # attack each node sequentially, one per step of the loop
                                for i in range (X_AXIS):
                                    for j in range (Y_AXIS):  
                                        for k in range (Z_AXIS):
                                            # Store the selected nodes
                                            selected_nodes.append([i, j, k])

                                            # Play the game with the current params
                                            games, lose, result, movements, time = play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details)

                                            # Update the attack file
                                            write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), x, y, str(position_index+1), str(movements), str(time), str(result), str(1), str(selected_nodes)], 'a')

                                            # Restart variables for next iterations
                                            selected_nodes = []

                            else: # attack multiple random neurons simultaneously
                                # Number of test repetitions indicated by n_exec
                                for current_exec in range(0, n_exec):

                                    # Generate random 3D coordinates for the current number of nodes
                                    for current_random in range(0, current_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                        # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
                                        while ([randX, randY, randZ] in selected_nodes):
                                            randX = np.random.randint(0, X_AXIS)
                                            randY = np.random.randint(0, Y_AXIS)
                                            randZ = np.random.randint(0, Z_AXIS)

                                        # Store the selected node
                                        selected_nodes.append([randX, randY, randZ])

                                    # Play the game with the current params
                                    
                                    #games=10
                                    #lose=1
                                    #result=True
                                    #movements=26
                                    #time=11111
                                    
                                    games, lose, result, movements, time = play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details)
                                    
                                    # movements (number of steps) from the current position of the mouse
                                    # we need to add the steps until reaching that position
                                    movements += position_index
                                    
                                    # Update the attack file
                                    write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), x, y, str(position_index+1), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                    # Restart variables for next iterations
                                    selected_nodes = []

                            #file_details.write("=== End games from position: ("+str(x)+","+str(y)+"), WIN: "+str(games-lose)+"\n\n\n")
                            #file_details.flush()
                            #file_summary.write("=== Games from position: ("+str(x)+","+str(y)+") with increment of: "+str(stim_value)+", NODES: "+str(current_nodes)+", PLAYED: "+str(games)+", WINNERS: "+str(games-lose)+" ===\n\n\n")
                            #file_summary.flush()

                    else: # SPO, SYB
                        selected_pairs_nodes = [] # pairs of nodes to be exchanged

                        # Number of test repetitions indicated by n_exec
                        for current_exec in range(0, n_exec):
                            if attack in ['SPO']:
                                # Generate random 3D coordinates for the current number of nodes -> pairs of nodes
                                for current_random in range(0, current_nodes):
                                    # Node 1 of the pair
                                    randX1 = np.random.randint(0, X_AXIS)
                                    randY1 = np.random.randint(0, Y_AXIS)
                                    randZ1 = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates for node 1
                                    while ([randX1, randY1, randZ1] in selected_nodes):
                                        randX1 = np.random.randint(0, X_AXIS)
                                        randY1 = np.random.randint(0, Y_AXIS)
                                        randZ1 = np.random.randint(0, Z_AXIS)

                                    selected_nodes.append([randX1, randY1, randZ1])

                                    # Node 2 of the pair
                                    randX2 = np.random.randint(0, X_AXIS)
                                    randY2 = np.random.randint(0, Y_AXIS)
                                    randZ2 = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates for node 2
                                    while ([randX2, randY2, randZ2] in selected_nodes):
                                        randX2 = np.random.randint(0, X_AXIS)
                                        randY2 = np.random.randint(0, Y_AXIS)
                                        randZ2 = np.random.randint(0, Z_AXIS)

                                    selected_nodes.append([randX2, randY2, randZ2])

                                    # Store the selected neuron
                                    selected_pairs_nodes.append([[randX1, randY1, randZ1], [randX2, randY2, randZ2]])

                                # Play the game with the current params
                                games, lose, result, movements, time = play_game_attacks_exchange(x, y, games, lose, selected_pairs_nodes, file_details)
                                
                                # movements (number of steps) from the current position of the mouse
                                # we need to add the steps until reaching that position
                                movements += position_index
                                    
                                # Update the attack file
                                write_to_csv_file(file_attack_name, [attack, current_nodes, x, y, str(position_index+1), str(movements), str(time), str(result), str(current_exec), str(selected_pairs_nodes)], 'a')

                                # Restart variables for next iterations
                                selected_nodes = []
                                selected_pairs_nodes = []

                            else:
                                # Generate random 3D coordinates for the current number of nodes
                                for current_random in range(0, current_nodes):

                                    randX = np.random.randint(0, X_AXIS)
                                    randY = np.random.randint(0, Y_AXIS)
                                    randZ = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates
                                    while ([randX, randY, randZ] in selected_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                    # Store the selected node
                                    selected_nodes.append([randX, randY, randZ])

                                # Play the game with the current params
                                games, lose, result, movements, time = play_game_attacks_mirror(x, y, games, lose, selected_nodes, file_details)
                                
                                # movements (number of steps) from the current position of the mouse
                                # we need to add the steps until reaching that position
                                movements += position_index
                                    
                                # Update the attack file
                                write_to_csv_file(file_attack_name, [attack, current_nodes, x, y, str(position_index+1), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                # Restart variables for next iterations
                                selected_nodes = []    

                        #file_details.write("=== End games from position: ("+str(x)+","+str(y)+"), WIN: "+str(games-lose)+"\n\n\n")
                        #file_details.flush()
                        #file_summary.write("=== Games from position: ("+str(x)+","+str(y)+"), NODES: "+str(current_nodes)+", PLAYED: "+str(games)+", WINNERS: "+str(games-lose)+" ===\n\n\n")
                        #file_summary.flush()
                
                # Increment the index used to label the position of the optimal path
                #position_index += 1
        #file_details.close()
        #file_summary.close()

In [ ]:
#attack_manager('SCA', 1, 1, 1)

In [ ]:
#attack_manager('FOR', 1, 1, 1)

In [ ]:
attack_manager('FLO', 5, 10, 10, [4,10,16,22])

In [ ]:
#attack_manager('JAM', 5, 10, 10)

In [ ]:
#attack_manager('SPO', 5, 10, 10)

In [ ]:
#attack_manager('SYB', 5, 10, 10)

In [20]:
######### JAMMING #########
'''
attack: string with the name of the attack
initial_node: index defining the number of the first attacked node
step_node: increment in the index of the attacked nodes used to adapt the size of the experiments
end_node: highest attacked index of a node
n_exec: number of executions of each particular combination of parameters
'''
def attack_manager(attack, initial_node, step_node, n_exec, number_positions, is_change_model):
    X_AXIS = 5
    Y_AXIS = 5
    Z_AXIS = 8 # number of filters
    X_MAZE = 7
    Y_MAZE = 7
    
    selected_nodes = [] # nodes selected to be attacked
    
    if attack in attacks_dict:
        dir_name = '/data/CNN/output/'
        
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            
        file_attack_name =  dir_name + 'attack' + attacks_dict[attack][0] + '.csv'
        file_details_name = dir_name + 'attack' + attacks_dict[attack][0] + '_details.txt'
        file_summary_name = dir_name + 'attack' + attacks_dict[attack][0] + '_summary.txt'

        # Open files
        file_details = open(file_details_name,"w")   
        file_summary = open(file_summary_name,"w")
        
        # Write header (overwrites the file)
        write_to_csv_file(file_attack_name, header_jamming, 'w')
        
        # Each number of neurons to consider
        for current_nodes in range(initial_node, attacks_dict[attack][1]+1, step_node):

            games=0
            lose=0

            if attack not in ['SPO', 'SYB']: # SCA, FOR, FLO, JAM
                x = 0
                y = 0 # the rat always starts in the first position
                
                # Each stimulation value
                for stim_value in attacks_dict[attack][2]:

                    if attack in ['JAM']: # attack multiple random neurons simultaneously
                        
                        if is_change_model:
                            for positions_attacked in range(1, number_positions+1):

                                # Number of test repetitions indicated by n_exec
                                for current_exec in range(0, n_exec):

                                    # Generate random 3D coordinates for the current number of nodes
                                    for current_random in range(0, current_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                        # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
                                        while ([randX, randY, randZ] in selected_nodes):
                                            randX = np.random.randint(0, X_AXIS)
                                            randY = np.random.randint(0, Y_AXIS)
                                            randZ = np.random.randint(0, Z_AXIS)

                                        # Store the selected node
                                        selected_nodes.append([randX, randY, randZ])

                                    # Play the game with the current params
                                    games, lose, result, movements, time = play_game_attacks_Jamming(x, y, games, lose, positions_attacked, is_change_model, selected_nodes, stim_value, file_details)

                                    # Update the attack file
                                    write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), str(positions_attacked), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                    # Restart variables for next iterations
                                    selected_nodes = []
                        else:
                            # Number of test repetitions indicated by n_exec
                            for current_exec in range(0, n_exec):

                                # Generate random 3D coordinates for the current number of nodes
                                for current_random in range(0, current_nodes):
                                    randX = np.random.randint(0, X_AXIS)
                                    randY = np.random.randint(0, Y_AXIS)
                                    randZ = np.random.randint(0, Z_AXIS)

                                    # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
                                    while ([randX, randY, randZ] in selected_nodes):
                                        randX = np.random.randint(0, X_AXIS)
                                        randY = np.random.randint(0, Y_AXIS)
                                        randZ = np.random.randint(0, Z_AXIS)

                                    # Store the selected node
                                    selected_nodes.append([randX, randY, randZ])

                                # Play the game with the current params
                                games, lose, result, movements, time = play_game_attacks_Jamming(x, y, games, lose, 27, is_change_model, selected_nodes, stim_value, file_details)

                                # Update the attack file
                                write_to_csv_file(file_attack_name, [attack, current_nodes, str(stim_value), str(27), str(movements), str(time), str(result), str(current_exec), str(selected_nodes)], 'a')

                                # Restart variables for next iterations
                                selected_nodes = []
                            

In [21]:
def append_to_csv_file(filename, line):
    # Update the attack file
    with open(filename, 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=';')
        writer.writerow(line)

In [ ]:
import time

output_file = '/data/CNN/output/execution_time.txt'

t = time.localtime()
current_time = time.strftime("%d %b %Y - %H:%M:%S", t)
append_to_csv_file(output_file, ["Init: "+str(current_time)])

attack_manager('JAM', 5, 10, 20, 27, False)
#attack_manager('JAM', 1, 1, 100, 27, False)

t = time.localtime()
current_time = time.strftime("%d %b %Y - %H:%M:%S", t)
append_to_csv_file(output_file, ["End: "+str(current_time)])

In [ ]:
## TEST FLO
X_AXIS = 5
Y_AXIS = 5
Z_AXIS = 8 # number of filters
selected_nodes = []

x = 0
y = 0
games = 0
lose = 0
stim_value = 0.9
file_details = ""

current_nodes = 10

for current_random in range(0, current_nodes):
    randX = np.random.randint(0, X_AXIS)
    randY = np.random.randint(0, Y_AXIS)
    randZ = np.random.randint(0, Z_AXIS)

    # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
    while ([randX, randY, randZ] in selected_nodes):
        randX = np.random.randint(0, X_AXIS)
        randY = np.random.randint(0, Y_AXIS)
        randZ = np.random.randint(0, Z_AXIS)

    # Store the selected node
    selected_nodes.append([randX, randY, randZ])
    
print(selected_nodes)

games, lose, result, movements, time, actions = play_game_attacks(x, y, games, lose, selected_nodes, stim_value, file_details)

print("success: ", str(result))
print("steps: ", str(movements))

selected_nodes = []

optimal_path_actions = ",down,right,right,up,right,right,right,down,down,left,left,down,left,left,left,down,down,down,right,right,up,right,right,right,down,right"
if optimal_path_actions == actions:
    print("iguales")
else:
    print("distintos")
    print(actions)

In [ ]:
## TEST JAM
X_AXIS = 5
Y_AXIS = 5
Z_AXIS = 8 # number of filters
selected_nodes = []

x = 0
y = 0
games = 0
lose = 0
positions_attacked = 3
stim_value = -1.0
file_details = ""

current_nodes = 5
is_change_model = False

for current_random in range(0, current_nodes):
    randX = np.random.randint(0, X_AXIS)
    randY = np.random.randint(0, Y_AXIS)
    randZ = np.random.randint(0, Z_AXIS)

    # Avoid duplicates -> iterate until finding a combination not existing in selected_nodes
    while ([randX, randY, randZ] in selected_nodes):
        randX = np.random.randint(0, X_AXIS)
        randY = np.random.randint(0, Y_AXIS)
        randZ = np.random.randint(0, Z_AXIS)

    # Store the selected node
    selected_nodes.append([randX, randY, randZ])
    
print(selected_nodes)

games, lose, result, movements, time = play_game_attacks_Jamming(x, y, games, lose, positions_attacked, is_change_model, selected_nodes, stim_value, file_details)

print("success: ", str(result))
#print("steps: ", str(movements))

selected_nodes = []

In [ ]:
optimal_path_actions = ",down,right,right,up,right,right,right,down,down,left,left,down,left,left,left,down,down,down,right,right,up,right,right,right,down,right"
if optimal_path_actions == actions:
    print("iguales")
else:
    print("distintos")
    print(actions)

#### Run CNN without attacks (same approach as for the attacks, but without masks)

In [ ]:
def generate_data_without_attacks(list_positions_attack, attack):
    header = ['attack', 'n_nodes', 'stim_value', 'X', 'Y', 'position', 'n_steps', 'n_exec', 'time', 'solution', 'attacked_nodes']
    
    dir_name = '/data/CNN/output/'
        
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
    file_name = dir_name+"original_CNN.csv"
    
    if attack == "JAM":
        write_to_csv_file(file_name, header_jamming, 'w')
    else:
        write_to_csv_file(file_name, header, 'w')

    # Each position of the input maze
    #for x in range (7):
    #    for y in range (7):
            
    #for position_index in range(0, len(list_coordinates_optimal_path)):
    for position_index in list_positions_attack:
        #x = position[0]
        #y = position[1]
        x = list_coordinates_optimal_path[position_index][0]
        y = list_coordinates_optimal_path[position_index][1]
            
        # Check if the current position of the maze is free (avoid walls)
        if maze[x][y]==1:
            games=0
            lose=0

            games, lose, result, movements, time = play_game_without_attacks(x, y, games, lose)
            
            # movements (number of steps) from the current position of the mouse
            # we need to add the steps until reaching that position
            movements += position_index
            
            if attack == "JAM":
                write_to_csv_file(file_name, ["original_CNN", str(0), str(0), str(0), str(movements), str(time), str(result), str(0), str([])], 'a')
            else:
                write_to_csv_file(file_name, ["original_CNN", 0, str(0), x, y, str(position_index+1), str(movements), 0, str(time), str(result), str([])], 'a')

#generate_data_without_attacks(list_positions_attack=[4,10,16,22], attack="FLO") # FLO
generate_data_without_attacks(list_positions_attack=[0], attack="JAM") # JAM